In [ ]:
%reset

# 1. Imports

In [148]:
import mne
import time
import os
import numpy as np

import custom_modules.file_handling as  fh
import custom_modules.sgeyesub_ptu as eye

import importlib
importlib.reload(fh)
importlib.reload(eye)

<module 'custom_modules.sgeyesub_ptu' from 'C:\\Users\\peter\\Documents\\Code\\master-thesis\\03_analysis\\custom_modules\\sgeyesub_ptu.py'>

# 2. Constants

In [149]:
data_path = 'D:/Diplomarbeit_data/eeg/'
subjects = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07' , 'A08', 'A09', 'A10']

mne.set_log_level('WARNING')

# 3. EOG derivative calculations

In [150]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='bad_ch_interpolated', dst_fldr='eye_derivations_added')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Calculating derivations for subject {subject}', end=' ')

    # Calculate the eye derivations and store them in the dst folder
    eye.calc_eye_derivations(src_path, dst_path, subject)

    print()

print(f'Finished eye derivations, took me {round(time.time()-start)} seconds...')

Calculating derivations for subject A01 
Calculating derivations for subject A02 
Calculating derivations for subject A03 
Calculating derivations for subject A04 
Calculating derivations for subject A05 
Calculating derivations for subject A06 
Calculating derivations for subject A07 
Calculating derivations for subject A08 
Calculating derivations for subject A09 
Calculating derivations for subject A10 
Finished eye derivations, took me 5 seconds...


# 4. LP filter of EOG channels

In [151]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='eye_derivations_added', dst_fldr='eyesub_prepared')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Highpass filtering for subject {subject}', end=' ')

    # Lowpass filters the eog channels with a cutoff of 5.0 Hz:
    eye.lp_filter_derivatives(src_path, dst_path, subject, export=True)

    print()

print(f'Finished eye derivations, took me {round(time.time()-start)} seconds...')

Highpass filtering for subject A01 
Highpass filtering for subject A02 
Highpass filtering for subject A03 
Highpass filtering for subject A04 
Highpass filtering for subject A05 
Highpass filtering for subject A06 
Highpass filtering for subject A07 
Highpass filtering for subject A08 
Highpass filtering for subject A09 
Highpass filtering for subject A10 
Finished eye derivations, took me 10 seconds...


# 5. Epoching

In [152]:
# Define markers of interest:
markers_of_interest = ['Blink', 'Rest' ,'Horz', 'Vert']

# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='eyesub_prepared', dst_fldr='eyesub_epoched')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Epoching for subject {subject}', end=' ')

    # Lowpass filters the eog channels with a cutoff of 5.0 Hz:
    eye.epoch_eye_paradigm(src_path, dst_path, subject, mrks=markers_of_interest, export=True)

    print()

print(f'Finished eye derivations, took me {round(time.time()-start)} seconds...')

Epoching for subject A01 Opening raw data file D:/Diplomarbeit_data/eeg/eyesub_prepared/A01_eye_lp_filtered_raw.fif...
    Range : 1130644 ... 1302243 =   5653.220 ...  6511.215 secs
Ready.
Reading 0 ... 171599  =      0.000 ...   857.995 secs...
Used Annotations descriptions: ['Blink', 'Break', 'Cue', 'Horz', 'Rest', 'Start', 'Vert']
Not setting metadata
54 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 54 events and 1601 original time points ...
0 bad epochs dropped
Overwriting existing file.

Epoching for subject A02 Opening raw data file D:/Diplomarbeit_data/eeg/eyesub_prepared/A02_eye_lp_filtered_raw.fif...
    Range : 7106 ... 159409 =     35.530 ...   797.045 secs
Ready.
Reading 0 ... 152303  =      0.000 ...   761.515 secs...
Used Annotations descriptions: ['Blink', 'Break', 'Cue', 'Horz', 'Rest', 'Start', 'Vert']
Not setting metadata
54 matching events found
No baseline correction applied
0 projection items a

# 6. Reject by visual inspection

In [50]:
# TODO

# 7. Detect saccades and blinks

In [142]:
# Load example epoch:
src_path = data_path + 'eyesub_prepared'
# There can be only one file  with matching conditions since we are splitting in folders:
f_name = [f for f in os.listdir(src_path) if ('A02' in f) and ('eye' in f)][0]

file = src_path + '/' + f_name
epochs = mne.io.read_raw(file, preload=True)


In [96]:
blink_fp_th = 100
blink_vert_fp_th = 150
blink_tp_th = 75e-6
blink_time_min = 0.025

saccade_fp_th = 100
saccade_tp_th = 10

In [107]:
# Find the median sign of the 10% highest peaks
ch_idx = [i for i, name in enumerate(epochs.ch_names) if name == 'EOGV'][0]
veog_blink_data = epochs['Blink'].get_data()[:,ch_idx,:]

# Get all occurences where the data exceeds the threshold:
veog_blink_mask = veog_blink_data > blink_tp_th
b = np.ones(int(blink_time_min*epochs.info['sfreq']))



In [108]:
from scipy.signal import filtfilt

res = filtfilt(b, 1, veog_blink_mask)
# res = np.convolve(veog_blink_mask,min_blink_samples, 'same')

In [105]:
res = np.apply_along_axis(lambda m: np.convolve(m, min_blink_samples, mode='same'), axis=1, arr=veog_blink_mask)

In [ ]:
# Detect blink if VEOG exceeds a threshold of 75µV for at least 25ms in a BLINK epoch:

ref_sig = EEG[channel_idx, :, label_mask]
peak_idxs = np.argsort(np.abs(ref_sig.ravel()))[::-1][:int(EEG.n_times*sum(label_mask)*0.01)]
peak_sign = np.median(np.sign(EEG[channel_idx, peak_idxs]))

    # Detect blinks as periods during which the EOG signal is outside the interval spanned by +/- threshold
blink_signal = (EEG[channel_idx, :] * peak_sign) > threshold
blink_signal[:, EEG.annotations.description != label] = 0

if t_extend > 0:
    # Apply a FIR filter to extend the blink signal
    b = np.ones(int(np.round(t_extend*EEG.info['sfreq'])))
    blink_signal = filtfilt(b, 1, blink_signal.astype(float))
    blink_signal = blink_signal > 0

blink_signal = np.reshape(blink_signal, (1, EEG.n_times, EEG.annotations.shape[0]))

return blink_signal, peak_sign

In [ ]:
# Load eye data:
src_path = data_path + 'bad_ch_interpolated'
# There can be only one file  with matching conditions since we are splitting in folders:
f_name = [f for f in os.listdir(src_path) if ('A02' in f) and ('eye' in f)][0]

file = src_path + '/' + f_name
raw = mne.io.read_raw(file, preload=True)

raw.info

In [ ]:
# Extract the EOG data:
eog_r = raw.get_data(picks='EOGL')
eog_l = raw.get_data(picks='EOGR')
eog_c = raw.get_data(picks='EOGC')

# Calculate EOG derivatives:
heog = eog_r - eog_l
veog = eog_c - (eog_r + eog_l) / 2
reog = (eog_c + eog_r + eog_l) / 3

# Create nchannels x ntimes matrix:
eog_derivatives = np.vstack((heog, veog, reog))

# Create an info file to later add the channels:
info = mne.create_info(['EOGH', 'EOGV', 'EOGRad'], 200.0, ch_types='eog')

# Create a raw array for the eog derivatives:
raw_derivatives = mne.io.RawArray(eog_derivatives, info, first_samp=raw.first_samp)

raw = raw.add_channels([raw_derivatives], force_update_info=True)

In [ ]:
raw.plot(duration=60, proj=False, n_channels=len(raw.ch_names), remove_dc=False, title=f'EOG derivatives added.')

In [ ]:
# Lowpass filter the eog derivatives:
raw = raw.copy().filter(l_freq=None, h_freq=5.0, picks=['eog'], method='iir')

In [ ]:
raw.plot(duration=60, proj=False, n_channels=len(raw.ch_names), remove_dc=False, title=f'EOG lowpass filtered.')

In [ ]:
def get_subset_of_dict(full_dict, keys_of_interest):
    return dict((k, full_dict[k]) for k in keys_of_interest if k in full_dict)

In [ ]:
# Extract epochs:
events_from_annot, event_dict = mne.events_from_annotations(raw)

#
markers_of_interest = ['Blink', 'Rest' ,'Horz', 'Vert']

event_dict_of_interest = get_subset_of_dict(event_dict, markers_of_interest)

# TODO select event ID's of interest, hand over dict for event_id to make it easier to extract them:
epochs = mne.Epochs(raw, events_from_annot, event_id=event_dict_of_interest, tmin=1.0, tmax=9.0, baseline=None, reject_by_annotation=True, preload=True, picks=['eeg', 'eog'])


In [ ]:
epochs['Rest'].plot(picks=['eeg', 'eog'])

In [ ]:
epochs.info